# PaddleOCR-VL 合同解析实践项目

本 Notebook 记录了一个完整的 PaddleOCR-VL 多语言合同解析实践流程，涵盖项目背景、目录结构、环境准备、数据构造、模型调用、测试验证以及最终效果展示。希望通过逐步演示，让读者能够快速在本地复现并扩展到自己的业务场景。

## 项目目标
- 自动解析中英文合同（扫描件或文件）并生成结构化 Markdown/JSON。
- 从 OCR 结构化结果中抽取甲乙方、签署日期、合同金额、生效与到期时间等关键信息。
- 提供命令行工具与 Gradio Demo，方便二次开发与展示。
- 附带单元测试与可复现的合成样本，便于在飞桨星河社区撰写实践文章。

## 系统架构概览
````text
输入文件 (PDF/图片)
    │
    ├──> PaddleOCRVL (文档预处理 + 版面分析 + VLM 识别)
    │
    ├── 结构化 Markdown/JSON
    │
    ├──> contract_extraction.extract_contract_fields
    │       └── 正则 + 表格解析
    │
    ├──> 关键信息 (party_a, party_b, amount, dates...)
    │
    ├──> CLI / Notebook / Gradio 展示
    │
    └──> 数据导出 (Markdown、JSON、可选入库)
````

## 目录结构说明
````text
applications/contract_ocr_vl/
├── README.md                    # 项目说明
├── app.py                       # 命令行入口
├── contract_extraction.py       # 关键信息抽取模块
├── contract_ocr_pipeline.py     # PaddleOCR-VL 调用封装
├── generate_synthetic_contracts.py # 合同样本生成脚本
├── gradio_app.py                # Web Demo
├── contract_ocr_demo.ipynb      # 本 Notebook
├── data/
│   └── synthetic_contracts/     # 自动生成的合同文本样本
└── tests/
    └── test_contract_extraction.py # 单元测试
````

In [ ]:
from pathlib import Path

project_root = Path('applications/contract_ocr_vl')
for path in sorted(project_root.glob('**/*')):
    if path.is_file():
        print(path)


## 环境准备
1. 推荐使用 Python 3.8+ 虚拟环境。
2. 安装 PaddleOCR 套件及演示需要的依赖：
   ```bash
   python -m pip install -U "paddleocr[doc-parser]" gradio pillow pytest
   ```
3. 若计划使用 vLLM/SGLang 后端，请执行 `paddleocr install_genai_server_deps vllm` 并启动 `paddleocr genai_server`。

In [ ]:
import platform, sys
import importlib

print('Python', sys.version)
print('Platform', platform.platform())
for pkg in ['paddleocr', 'gradio', 'pytest']:
    spec = importlib.util.find_spec(pkg)
    status = '已安装' if spec else '未安装'
    print(f'{pkg}: {status}')


## 运行单元测试 (保障抽取逻辑可用)
单元测试通过表格/段落示例验证 `extract_contract_fields` 的可靠性。

In [ ]:
import pytest
pytest.main(['applications/contract_ocr_vl/tests/test_contract_extraction.py'])


## 第一步：构造合成合同样本
脚本会按照模板分别生成中英文合同 Markdown，可按需开启 `create_images=True` 渲染 PNG。

In [ ]:
from shutil import rmtree

samples_dir = project_root / 'data' / 'notebook_samples'
if samples_dir.exists():
    rmtree(samples_dir)
samples_dir.mkdir(parents=True, exist_ok=True)

from applications.contract_ocr_vl.generate_synthetic_contracts import create_samples
create_samples(samples_dir, count=4, create_images=False)
sorted(str(p) for p in samples_dir.iterdir())


## 第二步：执行 OCR 流程并抽取关键信息
`run_contract_ocr` 会自动应用文档预处理、版面检测和 VLM 识别，并将输出保存到指定目录。

In [ ]:
ocr_outputs = project_root / 'data' / 'notebook_outputs'
if ocr_outputs.exists():
    rmtree(ocr_outputs)
ocr_outputs.mkdir(parents=True, exist_ok=True)

from applications.contract_ocr_vl.contract_ocr_pipeline import run_contract_ocr
result = run_contract_ocr(samples_dir, output_dir=ocr_outputs)
result['fields']


## 输出检查：生成的 JSON / Markdown 文件
每份文件都会保存结构化 JSON 与 Markdown，可用于后续入库或排版。

In [ ]:
sorted(str(p) for p in ocr_outputs.glob('*'))


## 解析第一份合同的 Markdown
以下展示第一份样本经过 PaddleOCR-VL 解析后的 Markdown 结果。

In [ ]:
from IPython.display import Markdown as IPyMarkdown

markdown_segments = result.get('markdown', [])
if markdown_segments:
    first = markdown_segments[0]
    if isinstance(first, dict):
        md_text = first.get('markdown_text') or first.get('markdown', '')
    else:
        md_text = first
    IPyMarkdown(md_text)
else:
    print('未获取到 Markdown 结果，可能是 OCR 未成功执行。')


## 命令行调用示例
Notebook 外也可以通过 CLI 快速体验：
```bash
python -m applications.contract_ocr_vl.app \
    --input applications/contract_ocr_vl/data/notebook_samples/CN-001.md \
    --output-dir tmp_outputs \
    --vl-rec-backend native
```
可根据需要替换为 `vllm-server` 并传入 `--vl_rec_server_url` 提升推理速度。

## Gradio Web Demo
若想制作图形界面，直接运行：
```bash
python applications/contract_ocr_vl/gradio_app.py
```
即可上传合同查看解析结果。部署到服务器后，可作为企业内部文档解析工具。

## 总结与下一步
- 通过 PaddleOCR-VL 能够在统一流程内完成文档预处理、版面分析和多模态识别。
- 结合正则与 Markdown 结构，可高效抽取合同关键信息，后续可接入数据库或知识库。
- 可进一步优化：
  1. 扩充 `_FIELD_PATTERNS` 支持更多语言与模板；
  2. 将结果写入向量数据库构建合同问答系统；
  3. 引入 Flask/FastAPI 对外提供解析服务；
  4. 对接真实扫描 PDF，验证不同硬件与推理后端的性能。